In [29]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [15]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [109]:
df=pd.read_csv('../../Product-Classification-tests/LSTM/products-data-minimised.csv')

In [110]:
df

,NameDescription,categories
0,tungsten solder pick handle solder pick pickin...,"Arts, Crafts & Sewing"
1,write right screen protector sony tc screen n...,Cell Phones & Accessories
2,casio men dbc databank digital watch disconti...,"Clothing, Shoes & Jewelry"
3,factoryreconditioned dewalt dwkr heavyduty am...,Tools & Home Improvement
4,energizer light twoway light feature bright ...,Health & Personal Care
...,...,...
44799,berkley trilene xl yard fishing line clear l...,Sports & Outdoors
44800,badger airbrush company basic spray gun set ea...,"Arts, Crafts & Sewing"
44801,fimo soft polymer clay ounces tropical green ...,"Arts, Crafts & Sewing"
44802,wilton icing color oz rose petal icing color ...,Grocery & Gourmet Food


In [113]:
df.categories.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
      dtype=object)

In [112]:
l=[]
for text in df.categories.unique():
    l.append(text)

for i in range(df['categories'].size):
    for j in range(18):
        if df["categories"][i]==l[j]:
            df["categories"][i]=j

In [88]:
catList=df['categories'].unique().tolist()

In [21]:
#I do aspire here to have balanced classes
num_of_categories = 1987
shuffledList=[None]*len(catList)
    
shuffled = df.reindex(np.random.permutation(df.index))
for cat in catList:
    shuffledList[catList.index(cat)] = shuffled[shuffled['categories'] == cat][:num_of_categories]

#e = shuffled[shuffled['categories'] == 'e'][:num_of_categories]
#b = shuffled[shuffled['categories'] == 'b'][:num_of_categories]
#t = shuffled[shuffled['categories'] == 't'][:num_of_categories]
#m = shuffled[shuffled['categories'] == 'm'][:num_of_categories]
concated = pd.concat(shuffledList, ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [65]:
#One-hot encode the lab

for cat in catList:
    concated.loc[concated['categories'] == cat, 'LABEL'] = catList.index(cat)

print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=18)
print(labels[:10])
if 'categories' in concated.keys():
    concated.drop(['categories'], axis=1)

23581    11
21435    10
18899     9
26917    13
16626     8
1228      0
24593    12
30625    15
898       0
28662    14
Name: LABEL, dtype: int64
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


,NameDescription,categories,LABEL
23581,mother goose nursery rhyme block humpty dumpty...,Toys & Games,11
21435,hella h series amber fog lamp kit passenger c...,Automotive,10
18899,monoprice cat punch keystone jack yellow high...,Electronics,9
26917,broannutone b contempora sliding door surface ...,Appliances,13
16626,keltec belt clip s p mm keltec belt clip s p m...,Sports & Outdoors,8
...,...,...,...
30820,pasco pressure guage psi part precision made...,Industrial & Scientific,15
7204,dewalt dw inch amp reversing end handle drill...,Tools & Home Improvement,3
3084,tekkeon eztalker et mini bluetooth headset inn...,Cell Phones & Accessories,1
4568,doodlebops deedee costume toddler size t doodl...,"Clothing, Shoes & Jewelry",2


In [127]:
X_train = df.loc[:21999, 'NameDescription'].values
y_train = df.loc[:21999, 'categories'].values
X_test = df.loc[22000:, 'NameDescription'].values
y_test = df.loc[22000:, 'categories'].values

from keras.preprocessing.text import Tokenizer
 
# create the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['NameDescription'])

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_train)
encoded_docs_test = tokenizer.texts_to_sequences(X_test)

from keras.utils import pad_sequences
# pad sequences
max_length = max([len(s.split()) for s in X_train])
X_train = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
X_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

y_test=y_test.astype('int')
y_train=y_train.astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [121]:
y_train

array([0, 1, 2, ..., 15, 0, 0], dtype=object)

In [119]:
X.shape

(35766, 130)

In [64]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [85]:
concated.loc[:21999,'LABEL'].values.shape

(6091,)

In [117]:
X_train = X[:21999]
y_train = df.loc[:21999,'categories'].values
X_test  = X[22000:]
y_test  = df.loc[22000:,'categories'].values
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))  


((21999, 130), (22000,), (13765, 130), (22803,))


In [122]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))  

((22000, 1239), (22000,), (22804, 1239), (22804,))


In [128]:
class clust():
    #def _load_data(self, sklearn_load_ds):
    def _load_data(self):
        #data = sklearn_load_ds
        #X = pd.DataFrame(data.data)
        self.X_train=X_train
        self.X_test=X_test
        self.y_train=y_train
        self.y_test=y_test
        #self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

    def __init__(self):
        self._load_data()
    
    @timeit
    def classify(self, model=LogisticRegression(random_state=42)):
        model.fit(self.X_train, self.y_train)
        y_pred = model.predict(self.X_test)
        print('Accuracy: {}'.format(accuracy_score(self.y_test, y_pred)))

    @timeit
    def Kmeans(self, output='add'):
        n_clusters = len(np.unique(self.y_train))
        clf = KMeans(n_clusters = n_clusters, random_state=42)
        clf.fit(self.X_train)
        y_labels_train = clf.labels_
        y_labels_test = clf.predict(self.X_test)
        if output == 'add':
            self.X_train['km_clust'] = y_labels_train
            self.X_test['km_clust'] = y_labels_test
        elif output == 'replace':
            self.X_train = y_labels_train[:, np.newaxis]
            self.X_test = y_labels_test[:, np.newaxis]
        else:
            raise ValueError('output should be either add or replace')
        
        print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))  
        return self

In [129]:
clust().Kmeans(output='replace').classify(model=SVC())

((22000, 1239), (22000,), (22804, 1239), (22804,))
'Kmeans'  8349.11 ms
Accuracy: 0.12028591475179792
'classify'  79375.21 ms


In [130]:
clust().classify()

Accuracy: 0.11230485879670234
'classify'  4097.40 ms


/Users/mouadlaayouni/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [131]:
clust().Kmeans(output='add').classify()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices